<a href="https://colab.research.google.com/github/VinitSR7/GLoVe-Implementation/blob/master/GLoVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
with open("preprocessedtext.txt", "rb") as fp:   
...   preprocessedText = pickle.load(fp)

- I will be using the "text" Data of the Amazon Fine Food Reviews Dataset.
- I preprocessed the Text saved it as preprocessedtext.txt, and using it as my dataset
- it is the list of sentencsa

In [11]:
type(preprocessedText), type(preprocessedText[0]), preprocessedText[0]

(list,
 str,
 'dogs loves chicken product china wont buying anymore hard find chicken products made usa one isnt bad good product wont take chances till know going china imports')

### Using Tf-Idf Vectorizer

In [62]:
tf_idf_vect = TfidfVectorizer(min_df=3)
X = tf_idf_vect.fit_transform(preprocessedText)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

some sample features(unique words in the corpus) ['aa', 'aaa', 'aaaa', 'aafco', 'ab', 'aback', 'abandon', 'abandoned', 'abandoning', 'abbott']


In [63]:
len(tf_idf_vect.get_feature_names())

21886

In [0]:
# Let's normalize the data first
from sklearn import preprocessing
X = preprocessing.normalize(X)

## Truncated-SVD

### Taking top features from TFIDF

In [0]:
feature_names = tf_idf_vect.get_feature_names()
idfscore = tf_idf_vect.idf_

In [91]:
#https://stackoverflow.com/questions/41445593/how-to-reverse-last-n-elements-of-list-in-python

idfscore_feat =zip(idfscore,feature_names) 
# Taking top 10000 features
top_features_sorted = sorted(idfscore_feat)
top_features_sorted = top_features_sorted[:-10001:-1]
len(top_features_sorted)

10000

In [92]:
top_words = [] 
for i in range(len(top_features_sorted)):
    top_words.append(top_features_sorted[i][1])
print(top_words[:10])

['zsweet', 'zp', 'zotz', 'zon', 'zombies', 'zola', 'zoji', 'ziyad', 'zipp', 'zinging']


### [5.2] Calulation of Co-occurrence matrix

In [0]:
cooccurrenceMatrix = np.zeros((10000,10000)) 
context_window = 5

In [94]:
from tqdm.notebook import tqdm
for sent in tqdm(preprocessedText):
    word_ = sent.split() 
    for index,word in enumerate(word_): 
        if word in top_words: 
            for j in range(max(index - context_window,0),min(index + context_window , len(word_) - 1) + 1):
                if word_[j] in top_words:
                    if word_[j] != word:
                        cooccurrenceMatrix[top_words.index(word)][top_words.index(word_[j])] += 1    
                        cooccurrenceMatrix[top_words.index(word)][top_words.index(word_[j])] += 1                

### Since GLoVe Vectors are of 384 dimensions, let's take 384 for our example too.

In [0]:
from sklearn.decomposition import TruncatedSVD

In [0]:
svd = TruncatedSVD(n_components= 384)
svd.fit(cooccurrenceMatrix)
transformed_data = svd.transform(cooccurrenceMatrix)

### That's it

###  Function that returns most similar words for a given word.

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(transformed_data) 
def find_similar_words(word, MAX = 20):
    if word not in top_words:
      print("Please input different word.")
      return
    word_vec = similarity_matrix[top_words.index(word)]  
    index = np.argsort(word_vec)  
    similar_words = [] 
    for x in index[:-(MAX+1):-1]:
      similar_words.append(top_words[x])
    print("{0} most similar words to {0} are given below".format(MAX,word))
    print(similar_words)

In [121]:
find_similar_words('admin', 10)

10 most similar words to 10 are given below
['admin', 'mop', 'eased', 'incur', 'ted', 'dices', 'backorder', 'dormant', 'pinky', 'overfed']


In [90]:
len(top_words)

5000

### Get numeric vector for a word

In [108]:
word = "borrow"
vector = transformed_data[top_words.index(word)]
len(vector)

384